In [ ]:
# coding:utf-8

from itertools import chain

import numpy as np
from joblib import Parallel, delayed

from pgmpy.estimators import ParameterEstimator
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianNetwork


class MaximumLikelihoodEstimator(ParameterEstimator):
    def __init__(self, model, data, **kwargs):

        if not isinstance(model, BayesianNetwork):
            raise NotImplementedError(
                "Maximum Likelihood Estimate is only implemented for BayesianNetwork"
            )
        elif set(model.nodes()) > set(data.columns):
            raise ValueError(
                f"Maximum Likelihood Estimator works only for models with all observed variables. Found latent variables: {model.latents}."
            )

        super(MaximumLikelihoodEstimator, self).__init__(model, data, **kwargs)

    def get_parameters(self, n_jobs=-1, weighted=False):

        parameters = Parallel(n_jobs=n_jobs, prefer="threads")(
            delayed(self.estimate_cpd)(node, weighted) for node in self.model.nodes()
        )

        return parameters


    def estimate_cpd(self, node, weighted=False):


        state_counts = self.state_counts(node, weighted=weighted)

        # if a column contains only `0`s (no states observed for some configuration
        # of parents' states) fill that column uniformly instead
        state_counts.values[:, (state_counts.values == 0).all(axis=0)] = 1

        parents = sorted(self.model.get_parents(node))
        parents_cardinalities = [len(self.state_names[parent]) for parent in parents]
        node_cardinality = len(self.state_names[node])

        # Get the state names for the CPD
        state_names = {node: list(state_counts.index)}
        if parents:
            state_names.update(
                {
                    state_counts.columns.names[i]: list(state_counts.columns.levels[i])
                    for i in range(len(parents))
                }
            )

        cpd = TabularCPD(
            node,
            node_cardinality,
            np.array(state_counts),
            evidence=parents,
            evidence_card=parents_cardinalities,
            state_names={var: self.state_names[var] for var in chain([node], parents)},
        )
        cpd.normalize()
        return cpd
    
    
    
    
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
data = pd.DataFrame(np.random.randint(low=0, high=2, size=(1000, 5)), columns=['A', 'B', 'C', 'D', 'E'])
model = BayesianNetwork([('A', 'B'), ('C', 'B'), ('C', 'D'), ('B', 'E')])
estimator = MaximumLikelihoodEstimator(model, data)